In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import lightgbm as lgb
import itertools
import pickle, gzip
import glob
from sklearn.preprocessing import StandardScaler
from tsfresh.feature_extraction import extract_features
np.warnings.filterwarnings('ignore')
import dask.dataframe as dd
import missingno as msno
from pandasql import sqldf
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import KFold
import matplotlib.gridspec as gridspec
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [2]:
#Always seed the randomness of this universe
np.random.seed(51)

In [3]:
%%time
pred1 = dd.read_csv('test_pred_37.csv')
pred2 = dd.read_csv('test_pred_38.csv')
pred1 = pred1.compute()
pred2 = pred2.compute()
print(pred1.shape,pred2.shape)

(3492890, 16) (3492890, 16)
CPU times: user 34.3 s, sys: 3.18 s, total: 37.5 s
Wall time: 8.07 s


In [4]:
%%time
pred1 = pred1.reset_index(drop=True)
pred2 = pred2.reset_index(drop=True)

CPU times: user 206 ms, sys: 260 ms, total: 466 ms
Wall time: 464 ms


In [5]:
pred1['object_id'].equals(pred2['object_id'])

True

In [6]:
%%time
test_ensemble = pd.DataFrame()

CPU times: user 286 µs, sys: 35 µs, total: 321 µs
Wall time: 326 µs


In [7]:
coef_lgb = 0.8
coef_mlp = 0.2

In [8]:
%%time
test_ensemble = pred1.iloc[:,1:]*coef_lgb + pred2.iloc[:,1:]*coef_mlp

CPU times: user 368 ms, sys: 520 ms, total: 887 ms
Wall time: 884 ms


In [9]:
print(test_ensemble.shape)

(3492890, 15)


In [10]:
test_ensemble['object_id'] = pred1['object_id']
print(test_ensemble.shape)

(3492890, 16)


In [11]:
test_ensemble = test_ensemble[list(pred1.columns)]

In [12]:
test_ensemble.tail()

,object_id,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
3492885,130787966,0.000173,0.365915,0.000309,0.188732,0.130489,0.000361,0.107619,0.000257,0.001176,0.016723,0.001749,0.185555,0.000232,0.000710,0.246124
3492886,130787971,0.000177,0.009494,0.000152,0.047120,0.237996,0.000218,0.015585,0.344344,0.000314,0.028925,0.000793,0.308762,0.000187,0.005932,0.239358
3492887,130787974,0.000611,0.025076,0.000140,0.825360,0.038703,0.000315,0.092123,0.000058,0.000150,0.001513,0.000373,0.013241,0.000196,0.002141,0.107618
3492888,130788053,0.000055,0.696708,0.000077,0.145715,0.006306,0.000108,0.011979,0.096263,0.000367,0.000378,0.000184,0.040249,0.000090,0.001519,0.174585
3492889,130788054,0.000221,0.095495,0.000249,0.412137,0.150778,0.000460,0.056253,0.000134,0.000610,0.023334,0.001154,0.257796,0.000592,0.000787,0.223972


In [13]:
%%time
test_ensemble.to_csv('test_pred_39.csv',index=False)

CPU times: user 1min 33s, sys: 1.38 s, total: 1min 34s
Wall time: 1min 34s


In [14]:
test_ensemble['object_id'].equals(pred1['object_id'])

True

In [15]:
!kaggle competitions submit -c PLAsTiCC-2018 -f test_pred_39.csv -m "Message"

100%|██████████████████████████████████████| 1.06G/1.06G [03:19<00:00, 5.73MB/s]
Successfully submitted to PLAsTiCC Astronomical Classification